In [1]:
# Візьміть датасет movielens і побудуйте модель матричної факторизації.
# У даній бібліотеці він має назву SVD. Підберіть найкращі параметри за допомогою крос-валідації,
# також поекспериментуйте з іншими алгоритмами розрахунків (SVD++, NMF) і оберіть той, який буде оптимальним.
# Підказки як саме побудувати дану модель ви знайдете в документації до даної бібліотеки.

In [2]:
pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 3.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp311-cp311-linux_x86_64.whl size=2505173 sha256=2459daa600dc670d2ccb55a98498108f0a7c166341f9d0c1bf95fce01b658fee
  Stored in directory: /root/.cache/pip/wheels/2a/8f/6e/7e2899163e2d85d8266daab4aa1cdabec7a6c56f83c015b5af
Successfully built scikit-surprise


In [10]:
from surprise import Dataset, Reader, SVD, SVDpp, NMF
from surprise.model_selection import train_test_split, cross_validate, GridSearchCV

# Завантажуємо вбудований датасет MovieLens 100k
data = Dataset.load_builtin('ml-100k')

In [7]:
print('Перші 5 рядків датасету:')
for row in data.raw_ratings[:5]:
  print(row)

Перші 5 рядків датасету:
('196', '242', 3.0, '881250949')
('186', '302', 3.0, '891717742')
('22', '377', 1.0, '878887116')
('244', '51', 2.0, '880606923')
('166', '346', 1.0, '886397596')


In [16]:
# Розділяємо датасет на навчальну та тестову вибірки (80% - навчальні дані, 20% - тестові).
# Для цього використовуємо функцію train_test_split, яка випадковим чином поділяє датасет.
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

# Створюємо різні моделі для рекомендацій:
# 1. SVD (Сингулярне розкладання) - стандартний алгоритм матричної факторизації.
# 2. SVD++ - модифікація SVD, яка додає покращену модель для врахування не тільки оцінок користувачів, а й інформації про їхній досвід.
# 3. NMF (Невід'ємна матрична факторизація) - альтернатива SVD, з іншими властивостями та вимогами до даних.
svd = SVD()
svdpp = SVDpp()
nmf = NMF()

# Виконуємо крос-валідацію для кожної з моделей (SVD, SVD++, NMF).
# Крос-валідація дозволяє оцінити ефективність моделі на різних підмножинах даних.
# Виводимо дві метрики: RMSE (корінь середньоквадратичної помилки) та MAE (середнє абсолютне відхилення).
# Параметр cv=5 означає, що дані будуть поділені на 5 частин для валідації.
print("SVD:")
svd_cv = cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)
print("------------------------------------------------")

print("\nSVD++:")
svdpp_cv = cross_validate(svdpp, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)
print("------------------------------------------------")

print("\nNMF:")
nmf_cv = cross_validate(nmf, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

SVD:
Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9277  0.9373  0.9336  0.9401  0.9328  0.9343  0.0042  
MAE (testset)     0.7301  0.7381  0.7346  0.7426  0.7358  0.7362  0.0041  
Fit time          1.43    1.36    1.33    1.34    1.41    1.37    0.04    
Test time         0.24    0.16    0.22    0.10    0.34    0.21    0.08    

SVD++:
Evaluating RMSE, MAE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9237  0.9282  0.9127  0.9178  0.9145  0.9194  0.0058  
MAE (testset)     0.7227  0.7259  0.7148  0.7231  0.7206  0.7214  0.0037  
Fit time          27.57   27.72   26.67   26.78   27.03   27.15   0.42    
Test time         5.09    5.34    4.37    5.32    4.41    4.90    0.43    

NMF:
Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    S

In [21]:
# Порівнюємо результати крос-валідації і вибираємо найкращу модель
# Найкраща модель буде та, яка має найменше значення RMSE і MAE.

# Список моделей та їх результатів
models = {
    'SVD': {'RMSE': sum(svd_cv['test_rmse']) / 5, 'MAE': sum(svd_cv['test_mae']) / 5},
    'SVD++': {'RMSE': sum(svdpp_cv['test_rmse']) / 5, 'MAE': sum(svdpp_cv['test_mae']) / 5},
    'NMF': {'RMSE': sum(nmf_cv['test_rmse']) / 5, 'MAE': sum(nmf_cv['test_mae']) / 5}
}

# Виведемо результати для кожної моделі
print("\nСередні результати крос-валідації:")
for model, scores in models.items():
    print(f"{model} - RMSE: {scores['RMSE']:.4f}, MAE: {scores['MAE']:.4f}")

# Обчислюємо середнє значення RMSE та MAE для кожної моделі і вибираємо модель з найменшим середнім значенням
best_model = min(models, key=lambda x: (models[x]['RMSE'] + models[x]['MAE']) / 2)

# Виводимо найкращу модель
print("\nНайкраща модель за комбінацією RMSE та MAE:", best_model)


Середні результати крос-валідації:
SVD - RMSE: 0.9343, MAE: 0.7362
SVD++ - RMSE: 0.9194, MAE: 0.7214
NMF - RMSE: 0.9638, MAE: 0.7574

Найкраща модель за комбінацією RMSE та MAE: SVD++


In [26]:
from surprise import SVD, SVDpp, NMF
from surprise.model_selection import GridSearchCV

def find_best_params_for_best_model(model_class, param_grid, data):
    """
    Вибір найкращих параметрів залежно від моделі.

    :param model_class: Клас моделі (SVD, SVD++, NMF)
    :param param_grid: Словник параметрів для GridSearch
    :param data: Датасет для тренування
    :return: Найкращі параметри та найкраща оцінка моделі
    """
    # Виконуємо GridSearchCV для пошуку найкращих параметрів
    grid_search = GridSearchCV(model_class, param_grid, measures=['RMSE', 'MAE'], cv=5)
    grid_search.fit(data)

    # Повертаємо найкращі параметри та результат
    return grid_search.best_params, grid_search.best_score

# Словник параметрів для кожної моделі
param_grids = {
    'SVD': {'n_factors': [50, 100, 150], 'reg_all': [0.01, 0.1, 0.2]},
    'SVD++': {'n_factors': [100], 'reg_all': [0.1]}, # зменшив кількість параметрів, бо розрахунок відбувається дуже довго
    'NMF': {'n_factors': [50, 100, 150], 'reg_all': [0.01, 0.1, 0.2]}
}

# Вибір найкращої моделі
model_classes = {
    'SVD': SVD,
    'SVD++': SVDpp,
    'NMF': NMF
}

# Знаходимо найкращу модель та її параметри
param_grid = param_grids[best_model]
model_class = model_classes[best_model]

# Знайдемо найкращі параметри для моделі
best_params, best_score = find_best_params_for_best_model(model_class, param_grid, data)

# Виведемо результати
if best_params:
    print(f"\nНайкращі параметри для моделі {best_model}: {best_params}")
    print(f"Найкраща оцінка для {best_model}: {best_score['rmse']:.4f}")


Найкращі параметри для моделі SVD++: {'rmse': {'n_factors': 100, 'reg_all': 0.1}, 'mae': {'n_factors': 100, 'reg_all': 0.1}}
Найкраща оцінка для SVD++: 0.9221


In [34]:
# Одержуємо навчальний набір даних
trainset = data.build_full_trainset()

# Вибираємо найкращі параметри для RMSE
best_params_rmse = best_params['rmse']

# Створюємо модель з отриманими параметрами
final_model = model_class(n_factors=best_params_rmse['n_factors'], reg_all=best_params_rmse['reg_all'])

# Тренуємо модель на всіх даних
final_model.fit(trainset)

# Тепер модель готова до прогнозів
print("Модель натренована і готова до прогнозів.")

Модель натренована і готова до прогнозів.


In [35]:
# Оцінка моделі на тестовому наборі
testset = trainset.build_testset()
predictions = final_model.test(testset)

# Виведення результатів
from surprise import accuracy

# Оцінка моделі
rmse = accuracy.rmse(predictions)
mae = accuracy.mae(predictions)

print(f"RMSE: {rmse:.4f}")
print(f"MAE: {mae:.4f}")

RMSE: 0.8288
MAE:  0.6588
RMSE: 0.8288
MAE: 0.6588


In [39]:
# Генерація рекомендацій для конкретного користувача

# Прогнозуємо оцінку для конкретного користувача та фільму
user_id = '196'
item_id = '242'

predicted_rating = final_model.predict(user_id, item_id)
print(f"Прогнозована оцінка для користувача {user_id} і фільму {item_id}: {predicted_rating.est:.2f}")

Прогнозована оцінка для користувача 196 і фільму 242: 3.84


In [40]:
# Отримуємо всі оцінки в датасеті
raw_ratings = data.raw_ratings

# Шукаємо реальну оцінку для користувача та фільму
real_rating = None
for rating in raw_ratings:
    if rating[0] == user_id and rating[1] == item_id:
        real_rating = rating[2]
        break

# Виведемо реальну оцінку, якщо вона є
if real_rating is not None:
    print(f"Реальна оцінка для користувача {user_id} і фільму {item_id}: {real_rating}")
else:
    print(f"Не знайдена реальна оцінка для користувача {user_id} і фільму {item_id}")

Реальна оцінка для користувача 196 і фільму 242: 3.0


In [42]:
# Рекомендація фільмів для користувача

# Отримуємо унікальні фільми
all_items = set(item for _, item, _, _ in data.raw_ratings)  # всі унікальні фільми

# Прогнозуємо оцінки для всіх фільмів
user_id = '196'
predictions = []

for item_id in all_items:
    # Прогнозуємо оцінку для кожного фільму
    pred = final_model.predict(user_id, item_id)
    predictions.append((item_id, pred.est))

# Сортуємо прогнози за оцінками
predictions.sort(key=lambda x: x[1], reverse=True)

# Отримуємо топ-5 рекомендованих фільмів
recommended_items = predictions[:5]

print(f"Топ-5 рекомендованих фільмів для користувача {user_id}:")
for movie_id, predicted_rating in recommended_items:
    print(f"Фільм ID {movie_id}: Прогнозована оцінка = {predicted_rating:.2f}")

Топ-5 рекомендованих фільмів для користувача 196:
Фільм ID 1449: Прогнозована оцінка = 4.42
Фільм ID 408: Прогнозована оцінка = 4.40
Фільм ID 169: Прогнозована оцінка = 4.40
Фільм ID 318: Прогнозована оцінка = 4.38
Фільм ID 64: Прогнозована оцінка = 4.35
